## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

## 데이터 불러오기

In [ ]:
Fold = 5

# k-fold 데이터
for i in range(Fold):
    
    j = i+1
    path1 = './SavedData/Training_Fold%d'%j
    path2 = './SavedData/Validation_Fold%d'%j
    c1 = 'Training_%d = np.array(pd.read_csv(path1, sep = ",", header = None))'%j
    c2 = 'Validation_%d = np.array(pd.read_csv(path2, sep = ",", header = None))'%j
    exec(c1)
    exec(c2)

# 학습 및 검증 데이터
Training_Label   = np.array(pd.read_csv('./SavedData/Training_Label', sep = ",", header = None))
Validation_Label = np.array(pd.read_csv('./SavedData/Validation_Label', sep = ",", header = None))

# 전체 학습용 데이터
Training_TotalData  = np.array(pd.read_csv('./SavedData/Training_TotalData', sep = ",", header = None))
Training_TotalLabel = np.array(pd.read_csv('./SavedData/Training_TotalLabel', sep = ",", header = None))

In [ ]:
Training_1.shape

## NN(Neural Network) hyperparameter 설정

In [ ]:
learningRate  = 0.0001
noOfNeuron    = 20
iteration     = 5000

## NN(Neural Network)의 구조(Architecture) 설계

In [ ]:
# input place holders
X = tf.placeholder(tf.float32, [None, Training_1.shape[1]])
Y = tf.placeholder(tf.float32, [None, 2])

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers
# Xavier : initializer=tf.contrib.layers.xavier_initializer()

# Input Layer
W2 = tf.get_variable("W2", shape=[Training_1.shape[1], noOfNeuron],
                     initializer=tf.variance_scaling_initializer())
b2 = tf.Variable(tf.random_normal([noOfNeuron]))
L2 = tf.nn.relu(tf.matmul(X, W2) + b2)

# Hidden Layer 1
W3 = tf.get_variable("W3", shape=[noOfNeuron, noOfNeuron],
                     initializer=tf.variance_scaling_initializer())
b3 = tf.Variable(tf.random_normal([noOfNeuron]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

# Hidden Layer 2
W4 = tf.get_variable("W4", shape=[noOfNeuron, noOfNeuron],
                     initializer=tf.variance_scaling_initializer())
b4 = tf.Variable(tf.random_normal([noOfNeuron]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

# Output Layer
W5 = tf.get_variable("W5", shape=[noOfNeuron, 2],
                     initializer=tf.variance_scaling_initializer())
b5 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L4, W5) + b5


# define cost/loss &optimizer
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## NN(Neural Network) 학습 및 검증

In [ ]:
Label = Training_Label

for i in range(Fold):
    
    s1= 'Data = Training_%d'%(i+1)
    exec(s1)

# train my model
# initialize
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for step in range(1, iteration+1):
    
        c, _ = sess.run([cost, optimizer], feed_dict = {X : Data, Y: Label})
        s2 = 'Validation_Acc_%d = sess.run(accuracy, feed_dict={X : Data, Y: Label})'%(i+1)
        exec(s2)

Accuracy_sum = 0

for j in range(Fold):
    
    s3 = 'Accuracy_sum += Validation_Acc_%d'%(j+1)
    exec(s3)
    
print('Fold 1: {:.5f} \nFold 2: {:.5f} \nFold 3: {:.5f} \nFold 4: {:.5f} \nFold 5: {:.5f} \n\n * Average accuracy : {:.5f}'
      .format(Validation_Acc_1, Validation_Acc_2, Validation_Acc_3, Validation_Acc_4, Validation_Acc_5, Accuracy_sum/Fold))

## 전체 학습 데이터로 NN 학습 및 모델 저장

In [ ]:
x_data = Training_TotalData
y_data = Training_TotalLabel

In [ ]:
# train my model
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(1, iteration+1):
    
    c, _ = sess.run([cost, optimizer], feed_dict = {X : x_data, Y: y_data})
    Acc = sess.run(accuracy, feed_dict={X : x_data, Y: y_data})
   
    if step % 1000 == 0:
            
        print('iteration {}'.format(step))    
        print('   Validation Acc: {:.5f}'.format(Acc))

In [ ]:
saver = tf.train.Saver()
saver.save(sess, './AI Model/ANN_model')

## NN model 불러와서 진단

텐서플로우 그래프 초기화 (아무것도 작성안했을 시 실행 X)

In [ ]:
# Tensorflow Graph 지울 때 사용
tf.reset_default_graph()

In [ ]:
# input place holders
X = tf.placeholder(tf.float32, [None, Training_1.shape[1]])
Y = tf.placeholder(tf.float32, [None, 2])

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers
# Xavier : initializer=tf.contrib.layers.xavier_initializer()

# Input Layer
W2 = tf.get_variable("W2", shape=[Training_1.shape[1], noOfNeuron],
                     initializer=tf.variance_scaling_initializer())
b2 = tf.Variable(tf.random_normal([noOfNeuron]))
L2 = tf.nn.relu(tf.matmul(X, W2) + b2)

# Hidden Layer 1
W3 = tf.get_variable("W3", shape=[noOfNeuron, noOfNeuron],
                     initializer=tf.variance_scaling_initializer())
b3 = tf.Variable(tf.random_normal([noOfNeuron]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

# Hidden Layer 2
W4 = tf.get_variable("W4", shape=[noOfNeuron, noOfNeuron],
                     initializer=tf.variance_scaling_initializer())
b4 = tf.Variable(tf.random_normal([noOfNeuron]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

# Output Layer
W5 = tf.get_variable("W5", shape=[noOfNeuron, 2],
                     initializer=tf.variance_scaling_initializer())
b5 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L4, W5) + b5


# define cost/loss &optimizer
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# 모델 불러오기 (그래프에 Variable w, b 불러옴)
sess = tf.Session()
saver=tf.train.Saver()
saver.restore(sess, './AI Model/ANN_model')

In [ ]:
# 전체 데이터 불러온 모델에 넣어 정확도 확인
sess.run(accuracy, feed_dict={X : Training_TotalData, Y: Training_TotalLabel})